In [1]:
from __future__ import print_function


from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import  LatentDirichletAllocation
from collections import Counter
import pandas as pd
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import numpy as np


raw_data=pd.read_csv("C:/Users/AyankoyaKayode/OneDrive - PRA Health Sciences/PRA/NLP/citeline_rare_disease.csv", header=1,
                     names = ['INCLD_CRTERIA','EXCLD_CRTERIA'], encoding="ISO-8859-1")

whole_data=pd.read_csv("C:/Users/AyankoyaKayode/OneDrive - PRA Health Sciences/PRA/NLP/citeline_rare_disease.csv", header=0,
                     encoding="ISO-8859-1")


whole_data_non=whole_data.dropna()

incld_list=whole_data_non['INCLD_CRTERIA'].tolist()

excld_list=whole_data_non['EXCLD_CRTERIA'].tolist()

In [2]:
n_samples = len(raw_data)
n_features = round(n_samples/6)
n_topics = len(whole_data.groupby('PRA_STUDY_ID')['PRA_STUDY_ID'].count())
n_top_words = 20

def print_top_words(model, feature_names, n_top_words):
    list_term_temp=[]
    #list_idx=[]

    for topic_idx, topic in enumerate(model.components_):
        #list_term_temp=[]
        #print("Topic #%d:" % topic_idx)
        #list_idx.append(topic_idx)
        #print(" ".join([feature_names[i]
        #                for i in topic.argsort()[:-n_top_words - 1:-1]]))
        for i in topic.argsort()[:-n_top_words -1:-1]:
            list_term_temp.append(feature_names[i])
            
        #list_term.append(list_term_temp)
    
    #dic=pd.DataFrame({'topic_index':list_idx, 'terms':list_term})
    #print()
    return list_term_temp

In [3]:
data_samples_incld = incld_list
data_samples_excld = excld_list


tf_vectorizer_incld = CountVectorizer(max_df=0.6, min_df=4, ngram_range=(2,2),
                                max_features=n_features,
                                stop_words='english')

tf_vectorizer_excld = CountVectorizer(max_df=0.6, min_df=4, ngram_range=(2,2),
                                max_features=n_features,
                                stop_words='english')

tf_incld = tf_vectorizer_incld.fit_transform(data_samples_incld)

tf_excld = tf_vectorizer_excld.fit_transform(data_samples_excld)

In [4]:
#from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
#vect=CountVectorizer(ngram_range=(1,1),stop_words='english')
#fin=vect.fit_transform(financedoc)
######pd.DataFrame(tf_incld.toarray(),columns=tf_vectorizer_excld.get_feature_names())

In [4]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda_incld = lda.fit(tf_incld)
lda_excld = lda.fit(tf_excld)


tf_feature_names_incld = tf_vectorizer_incld.get_feature_names()
incld_terms = print_top_words(lda_incld, tf_feature_names_incld, n_top_words)

tf_feature_names_incld = tf_vectorizer_excld.get_feature_names()
excld_terms = print_top_words(lda_excld, tf_feature_names_incld, n_top_words)


incld_terms_list = list(dict(Counter(incld_terms).most_common(100)))
excld_terms_list = list(dict(Counter(excld_terms).most_common(100)))

unique_incld = list(set(incld_terms_list) - set(excld_terms_list))
unique_excld = list(set(excld_terms_list) - set(incld_terms_list)) 


Fitting LDA models with tf features, n_samples=2081 and n_features=347...


C:\Users\AyankoyaKayode\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
C:\Users\AyankoyaKayode\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [6]:
#dict(Counter(excld_terms))
#dict(Counter(incld_terms))

sorting=np.argsort(lda.components_)[:,::-1]
features=np.array(tf_vectorizer_excld.get_feature_names())


In [24]:
import mglearn
dd1 = mglearn.tools.print_topics(topics=range(5), feature_names=features,sorting=sorting, topics_per_chunk=1, n_words=15)


topic 0       
--------      
situ cervix   
qt interval   
investigator jeopardize
acute chronic 
highly effective
female subjects
interfere evaluation
inflammatory bowel
defined investigator
clinically relevant
14 days       
congestive heart
12 months     
study months  
pregnant breast


topic 1       
--------      
cns metastases
breast feeding
known suspected
hepatic decompensation
history chronic
inflammatory bowel
suicide attempt
corticosteroid treatment
female subjects
liver disease 
participation investigational
subject history
laboratory results
defined investigator
laboratory abnormality


topic 2       
--------      
prior randomization
prior treatment
months prior  
blood pressure
uncontrolled hypertension
congestive heart
years prior   
investigational agent
medications known
basal cell    
immunodeficiency virus
condition opinion
cell skin     
active infection
gastrointestinal disorder


topic 3       
--------      
current diagnosis
28 days       
prior baseline
su

TypeError: 'NoneType' object is not subscriptable